In [ ]:
%load_ext autoreload

# 基于HS方法的实时光流计算

## 这个示例展示一个在AOP上运行的LK方法

调用接口：
- tianmoucv.proc.opticalflow.HS_optical_flow


In [ ]:
import os

train='/data/lyh/tianmoucData/tianmoucReconDataset/train/'
dirlist = os.listdir(train)
traindata = [train + e for e in dirlist]

val='/data/lyh/tianmoucData/tianmoucReconDataset/test/'
vallist = os.listdir(val)
valdata = [val + e for e in vallist]
key_list = []
print('---------------------------------------------------')
for sampleset in traindata:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e,end=" ")
        key_list.append(e)
print('---------------------------------------------------')
for sampleset in valdata:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e,end=" ")
        key_list.append(e)
        
all_data = valdata + traindata

# 光流计算

In [ ]:
%autoreload
import sys
import numpy as np
import torch
import matplotlib.pyplot as plt
import torch.nn.functional as F
import cv2, math
import torch.nn as nn

from tianmoucv.proc.reconstruct import TD_integration,SD_integration
from tianmoucv.isp import lyncam_raw_comp,demosaicing_npy,vizDiff
from tianmoucv.data import TianmoucDataReader
from tianmoucv.isp import SD2XY
from tianmoucv.proc.reconstruct import laplacian_blending
from tianmoucv.proc.opticalflow import interpolate_image,flow_to_image
from tianmoucv.proc.opticalflow import HS_optical_flow

from IPython.display import clear_output


speedUpRate = 1
count = 0
    
def images_to_video(frame_list,name,size=(640,320),Flip=True):
    fps = 25        
    ftmax = 1
    ftmin = 0
    out = cv2.VideoWriter(name,0x7634706d , fps, size)
    for ft in frame_list:
        ft = (ft-ftmin)/(ftmax-ftmin)
        ft[ft>1]=1
        ft[ft<0]=0
        ft2 = (ft*255).astype(np.uint8)
        out.write(ft2)
    out.release()


img_list = []
accumTime = 1
noiseThresh  = 6
lambda_of_HS = 8

for key in key_list:
    dataset = TianmoucDataReader(all_data,matchkey = 'test_exam_full3',MAXLEN=500)
    PSNR = 0
    img_list = []
    for index in range(len(dataset)):
        if index<= 5:
            print('rpogress:',index,'/',len(dataset))
            
            sample = dataset[index]
            F0 = sample['F0'].numpy()
            F1 = sample['F1'].numpy()
            tsdiff = sample['rawDiff']
            F0show = F0.copy()
            show_img = F0show.copy()
            print('Time len:',tsdiff.shape[1])
            
            for b in range((tsdiff.shape[1]-1)//accumTime):
                sd = 0
                td = 0
                TD = 0
                #积累几帧diff
                for t in range(accumTime):
                    threshed_tsdiff = tsdiff[:,b*accumTime+t,...].permute(1,2,0)
                    threshed_tsdiff[abs(threshed_tsdiff)<noiseThresh] = 0
                    SD = threshed_tsdiff[...,1:]
                    TD = threshed_tsdiff[...,0]
                    Ix,Iy= SD2XY(SD)
                    sd += torch.FloatTensor(np.stack([Ix,Iy],axis=0))
                    td += -(TD)

                # AOP预处理
                sd = sd/accumTime
                td = td.unsqueeze(0)
                td = F.interpolate(td.unsqueeze(0), size=sd.shape[1:], mode='bilinear').squeeze(0)

                # 计算OF
                rawflow = HS_optical_flow(sd,td,ifInterploted = True,epsilon = 1e-8,maxIteration = 20,labmda=lambda_of_HS,scales = 4)
                u = rawflow[0,:, :].numpy()
                v = rawflow[1,:, :].numpy()
                u = torch.Tensor(cv2.resize(u,(640,320))).unsqueeze(0)
                v = torch.Tensor(cv2.resize(v,(640,320))).unsqueeze(0)

                #可视化
                flow_show = flow_to_image(rawflow.permute(1,2,0).numpy())
                flow_show = torch.Tensor(cv2.resize(flow_show,(640,320)))/255.0
                flow_show = (flow_show*255).numpy().astype(np.uint8)

                mask = np.mean(flow_show,axis=-1) > 225
                flow_show[np.stack([mask]*3,axis=-1)]=0

                show_img = interpolate_image(show_img,u,v)

                td = tsdiff[0:1,(b)*accumTime:(b+1)*accumTime,...] 
                td = F.interpolate(td,(320,640),mode='bilinear')[0,0,...]
                rgb_td = vizDiff(td.cpu(),thresh=3,bg_color='black')
                tdiff_show = rgb_td.numpy() 
                
                sparsity = 4
                scale = 5
                for w in range(640//sparsity):
                    for h in range(320//sparsity):
                        x = int(w*sparsity)
                        y = int(h*sparsity)
                        u_ij = -u[0,y,x]
                        v_ij = -v[0,y,x]
                        color = flow_show[y,x,:]
                        color = tuple([int(e+20) for e in color])
                        if (u_ij**2+v_ij**2)>1:
                            cv2.arrowedLine(flow_show, (x,y), (int(x+u_ij*scale),int(y+v_ij*scale)), color,2, tipLength=0.15)

                tdiff_show_tensor = torch.Tensor(tdiff_show.copy())
                flow_show_tensor = torch.Tensor(flow_show)
                mask = torch.stack([torch.mean(flow_show_tensor,dim=-1)>0]*3,dim=-1)
                tdiff_show_tensor[mask] = flow_show_tensor[mask]/255.0
                tdiff_show_merge = tdiff_show_tensor.numpy()

                imshow1 = np.concatenate([flow_show/255.0,F0show],axis=1)
                imshow2 = np.concatenate([tdiff_show,tdiff_show_merge],axis=1)
                imshow = np.concatenate([imshow1,imshow2],axis=0)

                img_list.append(imshow)

                if b %5 ==0:
                    clear_output()
                    plt.figure(figsize=(6,3))
                    plt.axis('off') 
                    plt.imshow(imshow)
                    plt.show()
        else:
            clear_output()
            if not os.path.exists(output_dir):
                 os.mkdir(output_dir)
            output_name = os.path.join(output_dir,'OF_HS_viz_'+key+'.mp4')
            images_to_video(img_list,output_name,size=(640*2,320*2),Flip=True)
            break